# Photic Zone: Spectrophotometer

The photic zone notebooks concern sensor data from the surface to about 200 meters depth: The 'photic zone' is regularly illuminated by sunlight.


    
### Study site locations
    

We begin with three sites in the northeast Pacific: 
    

```
Site name               Lat               Lon
------------------      ---               ---
Oregon Offshore         44.37415          -124.95648
Oregon Slope Base       44.52897          -125.38966 
Axial Base              45.83049          -129.75326
```   

### to-do items
    
- optaa: rolling mean (days), use min for OA, smooth outliers for BA, animation, thin layer discrimination
- nitrate sample in relation to dark: Does an SME say there is more to do? 